In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from copy import deepcopy

#Carregando o dataset
hcc = pd.read_csv("hcc_dataset.csv", sep=",")

#Ajetiando os valores nulos
hcc.replace(np.nan, 'None', inplace=True) #Na tabela tem células com o valor None que ele interpreta como um np.nan, então precisamos garantir que ele vai entender isso como um valor válido
hcc.replace('?', np.nan, inplace=True) #As células vazias possuem uma '?', então aqui dizemos que essas células sõa NaN

for column in hcc.columns:
    #Convertendo os valores que são numéricos para float
    if hcc[column].dtype == 'object':
        try:
            if hcc[column].apply(lambda x: pd.to_numeric(x, errors='coerce')).notnull().any() and column != 'Nodules':
                hcc[column] = pd.to_numeric(hcc[column], errors='coerce')
        except ValueError:
            try:
                hcc[column] = pd.to_datetime(hcc[column], errors='coerce')
            except ValueError:
                pass

    #Tratando os valores de texto
    if(hcc[column].dtype == 'object'):
        hcc[column] = hcc[column].str.upper()

hcc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 50 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          165 non-null    object 
 1   Symptoms        147 non-null    object 
 2   Alcohol         165 non-null    object 
 3   HBsAg           148 non-null    object 
 4   HBeAg           126 non-null    object 
 5   HBcAb           141 non-null    object 
 6   HCVAb           156 non-null    object 
 7   Cirrhosis       165 non-null    object 
 8   Endemic         126 non-null    object 
 9   Smoking         124 non-null    object 
 10  Diabetes        162 non-null    object 
 11  Obesity         155 non-null    object 
 12  Hemochro        142 non-null    object 
 13  AHT             162 non-null    object 
 14  CRI             163 non-null    object 
 15  HIV             151 non-null    object 
 16  NASH            143 non-null    object 
 17  Varices         113 non-null    obj

In [2]:
def encoded(df, filename):
    columns_classification = { # Colunas que tem uma ordenação
        'PS': ['ACTIVE', 'RESTRICTED', 'AMBULATORY', 'SELFCARE', 'DISABLED'],
        'Encephalopathy': ['NONE', 'GRADE I/II', 'GRADE III/IV'],
        'Ascites': ['NONE', 'MILD', 'MODERATE/SEVERE']
    }
    
    # Transformando as variáveis categóricas em números
    for column in df:
        if hcc[column].dtype == 'object' and column != 'Nodules': # se for uma variável categórica, que não seja nodules, pois esse já é numérico
            if column in columns_classification:
                enc = OrdinalEncoder(categories=[columns_classification[column]]).set_output(transform="pandas") # vai codificar com ua ordem
            else:
                enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform="pandas") # vai criar colunas de 0's e 1's
            enc_transform = enc.fit_transform(df[[column]]) # faz a transformação da coluna codificada -> é uma nova tabela que precisa ser anexada no dataset
    
            # Aqui vamos considerar apenas as colunas importantes para que não seja anexada informação ambigua
            if not(column in columns_classification or column == 'Gender' or column == 'Class'):
                enc_transform = enc_transform.filter(regex='_YES$') # em colunas com Yes e No vamos considerar apenas o Yes, sendo ele 1 e o No 0
            elif column == 'Gender':
                enc_transform = enc_transform.filter(regex='_MALE$') # Como só tem 2 opção vamos considerar a coluna apenas dos homens, sendo para homem 1 e mulher 0
            elif column == 'Class':
                enc_transform = enc_transform.filter(regex='_LIVES$') # Como só tem 2 opção vamos considerar a coluna apenas das pessoas que sobreviveram, sendo para viver 1 e morrer 0
            df = df.drop(columns = [column]) # excluindo a coluna que originou a codificação, exenplo (Class)
            df = pd.concat([df, enc_transform], axis=1) # adicionando a codificação ao dataset
    
    df.to_csv(filename, index=False)
    return df

In [3]:
#Tratando os valores de texto -  colocando para UPPERCASE e colocando algum valor nas células vazias
encoded_hcc = deepcopy(hcc)
for column in encoded_hcc:
    if(encoded_hcc[column].dtype == 'object'):
        value = encoded_hcc[column].value_counts().idxmax() # para variáveis categóricas colocamos o valor mais frequente
    else:
        value = encoded_hcc[column].mean() # para valores numéricos colocamos a média
    encoded_hcc[column].replace(np.nan, value, inplace=True)
encoded_hcc = encoded(encoded_hcc, "default_imputation.csv")

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

encoded_knn = deepcopy(hcc)
label_encoders = {}

# Codificando variáveis categóricas
for column in encoded_knn:
    if(hcc[column].dtype == 'object'):
        label_encoders[column] = LabelEncoder()
        encoded_knn[column] = label_encoders[column].fit_transform(hcc[column].astype(str))
        if hcc[column].isna().any():
            nan_encoded = label_encoders[column].transform(['nan'])[0]
            encoded_knn[column] = encoded_knn[column].replace(nan_encoded, np.nan)
    
# Aplicando k-NN Imputation
knn_imputer = KNNImputer()
hcc_imputed_array = knn_imputer.fit_transform(encoded_knn)

# Convertendo de volta para DataFrame
hcc_imputed = pd.DataFrame(hcc_imputed_array, columns=encoded_knn.columns)

# Decodificando variáveis categóricas de volta aos valores originais
for column in encoded_knn:
    if(hcc[column].dtype == 'object'):
        hcc_imputed[column] = hcc_imputed[column].round().astype(int)
        hcc_imputed[column] = label_encoders[column].inverse_transform(hcc_imputed[column])

hcc_imputed = encoded(hcc_imputed, "knn_imputation.csv")

**DADOS ESCALONADOS**

In [5]:
def escalonamento(df, filename):
    colunas_numericas = df.select_dtypes(include=['int', 'float']).columns
    scaler = MinMaxScaler()
    colunas_nao_numericas = df.select_dtypes(exclude=['int', 'float']).columns
    dados_escalados_numericos=scaler.fit_transform(df[colunas_numericas])
    dados_escalados_numericos=pd.DataFrame(dados_escalados_numericos,columns=colunas_numericas)
    dados_escalados = pd.concat([dados_escalados_numericos, df[colunas_nao_numericas]], axis=1)
    df.to_csv(filename, index=False)

In [6]:
escalonamento(encoded_hcc, "esc_def_hcc.csv")
escalonamento(hcc_imputed, "esc_knn_hcc.csv")